In [1]:
% matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from scipy.stats import mode
from sklearn import datasets
iris = datasets.load_iris() 

In [2]:
def nearestPoints(knownData,testData,k):
    D1 = np.sum(np.square(knownData),axis = 1,keepdims=True)
    D2 = np.sum(np.square(testData),axis = 1,keepdims=True).T
    M = D1 + D2 - 2. * np.dot(knownData,testData.T)
    #print M
    return np.argsort(M,axis=0)[:k,:].T
    

def KNN(knownData, testData, knownClass, testClass,k):
    error = 0
    nearest = nearestPoints(knownData,testData,k)
    length = testData.shape[0]
    nearestClass = np.empty(length)
    #nearestClass = knownClass[np.argmax(np.apply_along_axis())
    for i in range(length):
        #nearestClass[i] = knownClass[np.argmax(np.bincount(nearest[i]))]
        nearestClass[i] = np.argmax(np.bincount([knownClass[a] for a in nearest[i]]))
    #for i in range(length):
        if(not(nearestClass[i] == testClass[i])):
           error += 1
    #print nearestClass
    return error

In [3]:
def splitData(data,dataClass,n):
    size = data.shape[0]
    arr = np.arange(size)
    np.random.shuffle(arr)
    mask1 = arr[:n]
    mask2 = arr[n:]
    return data[mask2],data[mask1],dataClass[mask2],dataClass[mask1]

In [4]:
rep = 5000
x = np.empty(rep)
for i in range(rep):
    knownData, testData, knownClass, testClass = splitData(iris.data,iris.target,50)
    x[i] = KNN(knownData,testData,knownClass,testClass,1)
print "Bląd średni wynosi: {}%".format((np.mean(x)/50) * 100)

Bląd średni wynosi: 4.4528%


In [5]:
digits = datasets.load_digits()
#print digits.data.shape
#print digits.target
rep = 50
x = np.empty(rep)
for i in range(rep):
    knownData, testData, knownClass, testClass = splitData(digits.data,digits.target,600)
    x[i] = KNN(knownData,testData,knownClass,testClass,5)
print x
print "Błąd średni wynosi: {}%".format((np.mean(x)/600) * 100)

[ 10.   7.  15.  12.  13.  10.   9.   6.   8.   9.   7.  10.  10.  14.  13.
   6.  11.   8.  12.  12.   8.   9.   7.   9.   8.  10.   9.  11.   7.  12.
  13.  11.   8.  10.  11.  17.   7.   7.   5.  12.  10.   7.  13.   9.  12.
  13.   7.   7.   9.  11.]
Błąd średni wynosi: 1.63666666667%


In [19]:
def crossValidation(data,dataClass,k):
    size = data.shape[0]
    arr = np.arange(size)
    np.random.shuffle(arr)
    err = 0
    n = int(size/10)
    for i in range(10):
        mask2 = arr[np.arange(i*n,min((i+1)*n,size))]
        mask1 = arr[np.concatenate((np.arange(i*n),np.arange((i+1)*n,size)))]
        #print "mask1: ",mask1.shape
        #print "mask2 :",mask2.shape
        #print mask1
        #print mask2
        err += KNN(data[mask1],data[mask2],dataClass[mask1],dataClass[mask2],k)
    return err

In [20]:
rep = 20
x = np.empty(rep)
t1 = time.time()
for i in range(rep):
    x[i] = crossValidation(iris.data,iris.target,1)
t2 = time.time()
print t2-t1,np.mean(x)

0.0683989524841 6.3


In [21]:
rep = 20
x = np.empty(rep)
t1 = time.time()
for i in range(rep):
    x[i] = crossValidation(digits.data,digits.target,1)
t2 = time.time()
print t2 - t1, np.mean(x)

5.35454893112 20.9
